# 0. Goals
- Do a lot of transformation
    - Encode categorical data using one hot encoding (v)
    - standardize the numerical columns (v)
    - Clustering data using K-Means (K=2), in order to create new features "label" need to be helped or not (v)
    - Evaluate the model before make prediction using classification model

# 1. Import Necessary Library

In [59]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [60]:
path_data_census = r'Output Data\census_data_ready_to_clustered.csv'

In [61]:
df_main = pd.read_csv(path_data_census)

In [62]:
df_main.head()

,age,workclass,census-weight,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,is_productive_age
0,39,State-gov,77516,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,1
1,50,Self-emp-not-inc,83311,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,1
2,38,Private,215646,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,1
3,53,Private,234721,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,1
4,28,Private,338409,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,1


In [63]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44114 entries, 0 to 44113
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   age                44114 non-null  int64 
 1   workclass          44114 non-null  object
 2   census-weight      44114 non-null  int64 
 3   education-num      44114 non-null  int64 
 4   marital-status     44114 non-null  object
 5   occupation         44114 non-null  object
 6   relationship       44114 non-null  object
 7   race               44114 non-null  object
 8   sex                44114 non-null  object
 9   capital-gain       44114 non-null  int64 
 10  capital-loss       44114 non-null  int64 
 11  hours-per-week     44114 non-null  int64 
 12  native-country     44114 non-null  object
 13  is_productive_age  44114 non-null  int64 
dtypes: int64(7), object(7)
memory usage: 4.7+ MB


# 2. Transform Data

## 2.1 One hot encoding categorical columns

In [64]:
# Get list of categorical columns

categorical_columns = df_main.select_dtypes(include=['object']).columns.tolist()

In [65]:
# Do encoding for categorical columns

encoder = OneHotEncoder(sparse_output=False)

one_hot_encoded = encoder.fit_transform(df_main[categorical_columns])

df_one_hot = pd.DataFrame(one_hot_encoded, columns= encoder.get_feature_names_out(categorical_columns))

df_main_encoded = pd.concat([df_main, df_one_hot], axis=1)

df_main_encoded = df_main_encoded.drop(categorical_columns, axis=1)

In [66]:
df_main_encoded.head()

,age,census-weight,education-num,capital-gain,capital-loss,hours-per-week,is_productive_age,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,native-country_others
0,39,77516,13,2174,0,40,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,50,83311,13,0,0,13,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,38,215646,9,0,0,40,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,53,234721,7,0,0,40,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,28,338409,13,0,0,40,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2.2 Standardize numerical columns

In [67]:
# Get list of numerical columns

numerical_columns = ['age', 'education-num', 'census-weight', 'capital-gain', 'capital-loss', 'hours-per-week']

In [68]:
# do standardize for numerical columns

standardizer = StandardScaler()

data_standardized = standardizer.fit_transform(df_main[numerical_columns])

df_standardized = pd.DataFrame(data_standardized, columns= standardizer.get_feature_names_out(numerical_columns) + '_std')

df_ready_clustered = pd.concat([df_main_encoded, df_standardized], axis=1)

df_ready_clustered = df_ready_clustered.drop(numerical_columns, axis=1)

In [69]:
df_ready_clustered.head()

,is_productive_age,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,workclass_others,...,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,native-country_others,age_std,education-num_std,census-weight_std,capital-gain_std,capital-loss_std,hours-per-week_std
0,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.037363,1.157101,-1.071458,0.582376,-0.22636,0.129531
1,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.839174,1.157101,-1.016533,-0.239673,-0.22636,-2.537017
2,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,-0.035529,-0.410095,0.237746,-0.239673,-0.22636,0.129531
3,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.057850,-1.193692,0.418540,-0.239673,-0.22636,0.129531
4,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.764448,1.157101,1.401300,-0.239673,-0.22636,0.129531


# 3. Build Clustering Model

In [70]:
kmeans = KMeans(n_clusters=2, random_state=42)

df_ready_clustered['label'] = kmeans.fit_predict(df_ready_clustered)

In [71]:
df_ready_clustered.head()

,is_productive_age,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,workclass_others,...,native-country_Vietnam,native-country_Yugoslavia,native-country_others,age_std,education-num_std,census-weight_std,capital-gain_std,capital-loss_std,hours-per-week_std,label
0,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.037363,1.157101,-1.071458,0.582376,-0.22636,0.129531,1
1,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.839174,1.157101,-1.016533,-0.239673,-0.22636,-2.537017,1
2,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-0.035529,-0.410095,0.237746,-0.239673,-0.22636,0.129531,0
3,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.057850,-1.193692,0.418540,-0.239673,-0.22636,0.129531,1
4,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-0.764448,1.157101,1.401300,-0.239673,-0.22636,0.129531,0


In [72]:
df_main['label'] = df_ready_clustered['label']

In [73]:
df_main.head()

,age,workclass,census-weight,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,is_productive_age,label
0,39,State-gov,77516,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,1,1
1,50,Self-emp-not-inc,83311,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,1,1
2,38,Private,215646,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,1,0
3,53,Private,234721,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,1,1
4,28,Private,338409,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,1,0


In [74]:
df_main[df_main['label'] == 1].describe()

,age,census-weight,education-num,capital-gain,capital-loss,hours-per-week,is_productive_age,label
count,22078.000000,2.207800e+04,22078.000000,22078.000000,22078.000000,22078.000000,22078.000000,22078.0
mean,45.920328,1.812935e+05,10.501449,1170.277878,163.080487,42.088867,0.936996,1.0
std,12.182638,9.770916e+04,2.664589,3593.295161,544.486546,8.853106,0.242975,0.0
min,17.000000,1.376900e+04,1.000000,0.000000,0.000000,1.000000,0.000000,1.0
25%,37.000000,1.139365e+05,9.000000,0.000000,0.000000,40.000000,1.000000,1.0
50%,45.000000,1.733070e+05,10.000000,0.000000,0.000000,40.000000,1.000000,1.0
75%,54.000000,2.265000e+05,13.000000,0.000000,0.000000,47.000000,1.000000,1.0
max,90.000000,1.226583e+06,16.000000,41310.000000,4356.000000,99.000000,1.000000,1.0


In [75]:
df_main[df_main['label'] == 0].describe()

,age,census-weight,education-num,capital-gain,capital-loss,hours-per-week,is_productive_age,label
count,22036.000000,2.203600e+04,22036.000000,22036.000000,22036.000000,22036.000000,22036.000000,22036.0
mean,31.040343,1.998485e+05,9.591078,96.383690,26.464876,35.281539,0.990016,0.0
std,10.829464,1.120180e+05,2.348067,698.789917,212.244821,10.175187,0.099421,0.0
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000,0.000000,0.0
25%,23.000000,1.244168e+05,9.000000,0.000000,0.000000,30.000000,1.000000,0.0
50%,29.000000,1.858205e+05,9.000000,0.000000,0.000000,40.000000,1.000000,0.0
75%,37.000000,2.501350e+05,10.000000,0.000000,0.000000,40.000000,1.000000,0.0
max,82.000000,1.490400e+06,16.000000,34095.000000,2444.000000,80.000000,1.000000,0.0
